# Google Drive | Google ドライブ> [Google Drive](https://en.wikipedia.org/wiki/Google_Drive)は、Googleによって開発されたファイルストレージおよび同期サービスです。>> > [Google Drive](https://en.wikipedia.org/wiki/Google_Drive) is a file storage and synchronization service developed by Google.このノートブックでは、`Google Drive`からドキュメントを読み込む方法について説明しています。現在、`Google Docs`のみがサポートされています。> This notebook covers how to load documents from `Google Drive`. Currently, only `Google Docs` are supported.## Prerequisites | 前提条件1. Google Cloudのプロジェクトを新規作成するか、既存のプロジェクトを使用してください   > Create a Google Cloud project or use an existing project2. [Google Drive API](https://console.cloud.google.com/flows/enableapi?apiid=drive.googleapis.com)を有効にする   > Enable the [Google Drive API](https://console.cloud.google.com/flows/enableapi?apiid=drive.googleapis.com)3. [デスクトップアプリの認証情報を承認する](https://developers.google.com/drive/api/quickstart/python#authorize_credentials_for_a_desktop_application)   > [Authorize credentials for desktop app](https://developers.google.com/drive/api/quickstart/python#authorize_credentials_for_a_desktop_application)4. `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib` コマンドを実行して、Google API Python クライアント、Google 認証 HTTPLib2、Google 認証 OAuthLib をアップグレードしてください。   > `pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib`## 🧑 Instructions for ingesting your Google Docs data | 🧑 Google Docsのデータを取り込むための指示デフォルトでは、`GoogleDriveLoader`は`credentials.json`ファイルを`~/.credentials/credentials.json`にあると想定していますが、これは`credentials_path`キーワード引数を使用して変更することができます。`token.json`についても同様で、`token_path`を使用して変更できます。`token.json`は、ローダーを初めて使用する時に自動的に作成されることに注意してください。> By default, the `GoogleDriveLoader` expects the `credentials.json` file to be `~/.credentials/credentials.json`, but this is configurable using the `credentials_path` keyword argument. Same thing with `token.json` - `token_path`. Note that `token.json` will be created automatically the first time you use the loader.GoogleDriveLoaderを初めて使用する際には、ブラウザで同意画面が表示されます。もし同意画面が表示されずに`RefreshError`が発生した場合は、`GoogleDriveLoader`コンストラクタの呼び出しで`credentials_path`を使用しないでください。代わりに、そのパスを`GOOGLE_APPLICATION_CREDENTIALS`環境変数に設定してください。> The first time you use GoogleDriveLoader, you will be displayed with the consent screen in your browser. If this doesn't happen and you get a `RefreshError`, do not use `credentials_path` in your `GoogleDriveLoader` constructor call. Instead, put that path in a `GOOGLE_APPLICATION_CREDENTIALS` environmental variable.`GoogleDriveLoader`は、Google DocsのドキュメントIDのリストまたはフォルダIDからロードすることができます。フォルダIDとドキュメントIDは、URLから取得することができます。> `GoogleDriveLoader` can load from a list of Google Docs document ids or a folder id. You can obtain your folder and document id from the URL:* フォルダ: https://drive.google.com/drive/u/0/folders/1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5 -> フォルダIDは `"1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5"` です  > Folder: https://drive.google.com/drive/u/0/folders/1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5 -> folder id is `"1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5"`* ドキュメント: https://docs.google.com/document/d/1bfaMQ18\_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw/edit -> ドキュメントIDは「1bfaMQ18\_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw」です。  > Document: https://docs.google.com/document/d/1bfaMQ18\_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw/edit -> document id is `"1bfaMQ18_i56204VaQDVeAFpqEijJTgvurupdEDiaUQw"`

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from langchain.document_loaders import GoogleDriveLoader

In [ ]:
loader = GoogleDriveLoader(
    folder_id="1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5",
    token_path="/path/where/you/want/token/to/be/created/google_token.json",
    # Optional: configure whether to recursively fetch files from subfolders. Defaults to False.
    recursive=False,
)

In [ ]:
docs = loader.load()

`folder_id`を渡すと、デフォルトではドキュメント、シート、PDFの全てのファイルタイプが読み込まれます。この挙動を変更するには、`file_types`引数を指定することができます。> When you pass a `folder_id` by default all files of type document, sheet and pdf are loaded. You can modify this behaviour by passing a `file_types` argument

In [ ]:
loader = GoogleDriveLoader(
    folder_id="1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5",
    file_types=["document", "sheet"],
    recursive=False,
)

## Passing in Optional File Loaders | オプショナルなファイルローダーを渡すGoogle DocsやGoogle Sheets以外のファイルを処理する際には、オプショナルなファイルローダーを`GoogleDriveLoader`に渡すと便利です。ファイルローダーを渡した場合、Google DocsやGoogle SheetsのMIMEタイプでないドキュメントにそのファイルローダーが使用されます。ここに、ファイルローダーを使用してGoogle DriveからExcelドキュメントをロードする方法の例を示します。> When processing files other than Google Docs and Google Sheets, it can be helpful to pass an optional file loader to `GoogleDriveLoader`. If you pass in a file loader, that file loader will be used on documents that do not have a Google Docs or Google Sheets MIME type. Here is an example of how to load an Excel document from Google Drive using a file loader.

In [ ]:
from langchain.document_loaders import GoogleDriveLoader, UnstructuredFileIOLoader

In [ ]:
file_id = "1x9WBtFPWMEAdjcJzPScRsjpjQvpSo_kz"
loader = GoogleDriveLoader(
    file_ids=[file_id],
    file_loader_cls=UnstructuredFileIOLoader,
    file_loader_kwargs={"mode": "elements"},
)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

以下のパターンを使用して、ファイルとGoogle Docs/Sheetsが混在しているフォルダも処理することができます：> You can also process a folder with a mix of files and Google Docs/Sheets using the following pattern:

In [ ]:
folder_id = "1asMOHY1BqBS84JcRbOag5LOJac74gpmD"
loader = GoogleDriveLoader(
    folder_id=folder_id,
    file_loader_cls=UnstructuredFileIOLoader,
    file_loader_kwargs={"mode": "elements"},
)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

## Extended usage | 拡張利用外部コンポーネントである `langchain-googledrive` は、Google Driveの複雑さを管理できます。これは `langchain.document_loaders.GoogleDriveLoader` と互換性があり、その代わりに使用できます。> An external component can manage the complexity of Google Drive : `langchain-googledrive`> It's compatible with the ̀`langchain.document_loaders.GoogleDriveLoader` and can be used> in its place.コンテナと互換性を持たせるために、認証では環境変数 `GOOGLE_ACCOUNT_FILE` をクレデンシャルファイル（ユーザーまたはサービス用）として使用します。> To be compatible with containers, the authentication uses an environment variable `̀GOOGLE_ACCOUNT_FILE` to credential file (for user or service).

In [ ]:
!pip install langchain-googledrive

In [ ]:
folder_id = "root"
# folder_id='1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5'

In [ ]:
# Use the advanced version.
from langchain_googledrive.document_loaders import GoogleDriveLoader

In [ ]:
loader = GoogleDriveLoader(
    folder_id=folder_id,
    recursive=False,
    num_results=2,  # Maximum number of file to load
)

デフォルトでは、これらのmime-typeを持つファイルはすべて`Document`に変換可能です。> By default, all files with these mime-type can be converted to `Document`.* テキスト/テキスト  > text/text* オリジナル: text/plain  > text/plain* text/html  > text/html* テキスト/CSV  > text/csv* <基準>  オリジナル:  text/markdown  > text/markdown* image/png  > image/png* image/jpeg  > image/jpeg* application/epub+zip  > application/epub+zip* application/pdf  > application/pdf* application/rtf  > application/rtf* application/vnd.google-apps.document (GDoc)  > application/vnd.google-apps.document (GDoc)* application/vnd.google-apps.presentation (Gスライド)  > application/vnd.google-apps.presentation (GSlide)* application/vnd.google-apps.spreadsheet (GSheet)  > application/vnd.google-apps.spreadsheet (GSheet)* application/vnd.google.colaboratory（ノートブック colab）  > application/vnd.google.colaboratory (Notebook colab)* application/vnd.openxmlformats-officedocument.presentationml.presentation (PPTX)  > application/vnd.openxmlformats-officedocument.presentationml.presentation (PPTX)* application/vnd.openxmlformats-officedocument.wordprocessingml.document (DOCX)  > application/vnd.openxmlformats-officedocument.wordprocessingml.document (DOCX)これを更新またはカスタマイズすることが可能です。`GDriveLoader`のドキュメントをご覧ください。> It's possible to update or customize this. See the documentation of `GDriveLoader`.ただし、対応するパッケージをインストールする必要があります。> But, the corresponding packages must be installed.

In [ ]:
!pip install unstructured

In [ ]:
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

### Customize the search pattern | 検索パターンをカスタマイズするGoogle [`list()`](https://developers.google.com/drive/api/v3/reference/files/list) APIと互換性のあるすべてのパラメータが設定可能です。> All parameter compatible with Google [`list()`](https://developers.google.com/drive/api/v3/reference/files/list)> API can be set.Googleリクエストの新しいパターンを指定するには、`PromptTemplate()`を使用できます。プロンプトの変数はコンストラクタで`kwargs`を使って設定できます。いくつかの事前フォーマットされたリクエストが提案されています（`{query}`, `{folder_id}`, および/または `{mime_type}`を使用）：> To specify the new pattern of the Google request, you can use a `PromptTemplate()`.> The variables for the prompt can be set with `kwargs` in the constructor.> Some pre-formated request are proposed (use `{query}`, `{folder_id}` and/or `{mime_type}`):ファイルを選択するための基準をカスタマイズすることができます。事前に定義されたフィルターのセットが提案されています：> You can customize the criteria to select the files. A set of predefined filter are proposed:| テンプレート                               | 説明                                                           || -------------------------------------- | --------------------------------------------------------------------- || gdrive-all-in-folder                   | `folder_id`から互換性のあるすべてのファイルを返します                        || gdrive-query                           | すべてのドライブで`query`を検索します                                          || gdrive-by-name                         | `query`という名前のファイルを検索します                                        || gdrive-query-in-folder                 | `folder_id`（および`recursive=true`の場合はサブフォルダーも含む）で`query`を検索します  || gdrive-mime-type                       | 特定の`mime_type`を検索します                                         || gdrive-mime-type-in-folder             | `folder_id`で特定の`mime_type`を検索します                          || gdrive-query-with-mime-type            | 特定の`mime_type`を使用して`query`を検索します                            || gdrive-query-with-mime-type-and-folder | 特定の`mime_type`を使用して、かつ`folder_id`内で`query`を検索します         |> | template                               | description                                                           |> | -------------------------------------- | --------------------------------------------------------------------- |> | gdrive-all-in-folder                   | Return all compatible files from a `folder_id`                        |> | gdrive-query                           | Search `query` in all drives                                          |> | gdrive-by-name                         | Search file with name `query`                                        |> | gdrive-query-in-folder                 | Search `query` in `folder_id` (and sub-folders if `recursive=true`)  |> | gdrive-mime-type                       | Search a specific `mime_type`                                         |> | gdrive-mime-type-in-folder             | Search a specific `mime_type` in `folder_id`                          |> | gdrive-query-with-mime-type            | Search `query` with a specific `mime_type`                            |> | gdrive-query-with-mime-type-and-folder | Search `query` with a specific `mime_type` and in `folder_id`         |

In [ ]:
loader = GoogleDriveLoader(
    folder_id=folder_id,
    recursive=False,
    template="gdrive-query",  # Default template to use
    query="machine learning",
    num_results=2,  # Maximum number of file to load
    supportsAllDrives=False,  # GDrive `list()` parameter
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

あなたは自分のパターンをカスタマイズできます。> You can customize your pattern.

In [ ]:
from langchain.prompts.prompt import PromptTemplate

loader = GoogleDriveLoader(
    folder_id=folder_id,
    recursive=False,
    template=PromptTemplate(
        input_variables=["query", "query_name"],
        template="fullText contains '{query}' and name contains '{query_name}' and trashed=false",
    ),  # Default template to use
    query="machine learning",
    query_name="ML",
    num_results=2,  # Maximum number of file to load
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

変換はマークダウン形式で行うことができます：> The conversion can manage in Markdown format:* 箇条書き  > bullet* リンク  > link* テーブル  > table* タイトル  > titles`return_link`属性を`True`に設定すると、リンクがエクスポートされます。> Set the attribut `return_link` to `True` to export links.#### Modes for GSlide and GSheet | GSlideとGSheetのモードパラメータ mode は異なる値を受け入れます：> The parameter mode accepts different values:* 「document」: 各ドキュメントの本文を返します  > "document": return the body of each document* 'snippets': 各ファイルの説明を返します（Googleドライブのファイルのメタデータに設定されています）。  > "snippets": return the description of each file (set in metadata of Google Drive files).`gslide_mode` パラメーターは、異なる値を受け入れます：> The parameter `gslide_mode` accepts different values:* "single" : 1つのドキュメントに\<PAGE BREAK>  > "single" : one document with \<PAGE BREAK>* 「slide」: スライド1枚につきドキュメント1つ  > "slide" : one document by slide* 「elements」: 各要素に対して一つのドキュメント。  > "elements" : one document for each elements.

In [ ]:
loader = GoogleDriveLoader(
    template="gdrive-mime-type",
    mime_type="application/vnd.google-apps.presentation",  # Only GSlide files
    gslide_mode="slide",
    num_results=2,  # Maximum number of file to load
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

`gsheet_mode` パラメータは、異なる値を受け入れます：> The parameter `gsheet_mode` accepts different values:* "single"：一行ごとに一つのドキュメントを生成する  > `"single"`: Generate one document by line* `"elements"` : マークダウン配列と`<PAGE BREAK>`タグが含まれる一つのドキュメント。  > `"elements"` : one document with markdown array and \<PAGE BREAK> tags.

In [ ]:
loader = GoogleDriveLoader(
    template="gdrive-mime-type",
    mime_type="application/vnd.google-apps.spreadsheet",  # Only GSheet files
    gsheet_mode="elements",
    num_results=2,  # Maximum number of file to load
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")

### Advanced usage | 高度な利用方法すべてのGoogleファイルにはメタデータ内に'description'（説明）フィールドがあります。このフィールドは、ドキュメントの要約やその他のインデックス付けされたタグを記録するために使用できます（`lazy_update_description_with_summary()` メソッドを参照してください）。> All Google File have a 'description' in the metadata. This field can be used to memorize a summary of the document or others indexed tags (See method `lazy_update_description_with_summary()`).`mode="snippet"`を使用する場合、本文には説明のみが使用されます。それ以外の場合、`metadata['summary']`がフィールドとして使用されます。> If you use the `mode="snippet"`, only the description will be used for the body. Else, the `metadata['summary']` has the field.時には、特定のフィルターを用いてファイル名から情報を抽出し、特定の基準を満たすファイルを選ぶことができます。フィルターを使用してみてください。> Sometime, a specific filter can be used to extract some information from the filename, to select some files with specific criteria. You can use a filter.時々、多くのドキュメントが返されることがあります。すべてのドキュメントを同時にメモリ上に保持する必要はありません。一度に1つのドキュメントを取得するために、メソッドの遅延バージョンを使用できます。再帰的な検索の代わりに複雑なクエリを使用する方がより良いです。`recursive=True`を有効にした場合、各フォルダに対してクエリを適用する必要があります。> Sometimes, many documents are returned. It's not necessary to have all documents in memory at the same time. You can use the lazy versions of methods, to get one document at a time. It's better to use a complex query in place of a recursive search. For each folder, a query must be applied if you activate `recursive=True`.

In [ ]:
import os

loader = GoogleDriveLoader(
    gdrive_api_file=os.environ["GOOGLE_ACCOUNT_FILE"],
    num_results=2,
    template="gdrive-query",
    filter=lambda search, file: "#test" not in file.get("description", ""),
    query="machine learning",
    supportsAllDrives=False,
)
for doc in loader.load():
    print("---")
    print(doc.page_content.strip()[:60] + "...")